In [18]:
import pandas as pd
import mlflow as mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from urllib.parse import urlparse # For parsing URLs  - used to parse the MLflow experiment URL
from sklearn.datasets import fetch_california_housing


In [19]:
fetch_california_housing()

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [20]:
# Creating pandas dataframe from California housing dataset
df = pd.DataFrame(fetch_california_housing().data, columns=fetch_california_housing().feature_names)
df['MedHouseVal'] = fetch_california_housing().target
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [21]:
# splitting the data into features (X) and target (y)

X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']   


# Splitting the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


signature = infer_signature(X_train, y_train)   #generate a model signature that describes the input and output schema of machine learning model


# Define the hyperparameters for GridSearchCV - for tuning hyperparameters on the RandomForestRegressor

param_grid = {
    'n_estimators': [100, 200, 300],
   'max_depth': [None, 5, 10],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4]
}



## Hyperparameter tuning using GridSearchCV Function
def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1,verbose=1 , scoring="neg_mean_square_error")     #n_jobs=-1 means use all processors
    grid_search.fit(X_train, y_train)
    return grid_search


#  Start MLFlow Experiments

with mlflow.start_run() as run:
    # Hyperparameter tunning  
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Log parameters and metrics
    mlflow.log_param("n_estimators", best_model.n_estimators)
    mlflow.log_param("max_depth", best_model.max_depth)
    mlflow.log_param("min_samples_split", best_model.min_samples_split)
    mlflow.log_param("min_samples_leaf", best_model.min_samples_leaf)
    mlflow.log_metric("mse", mse)


    # Tracking url
    mlflow.set_tracking_uri("http://localhost:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


    #If there is a url server,  register the model, if not save as a file localy
    if tracking_url_type_store!= "file":
        mlflow.sklearn.load_model(best_model, "model", registered_model_name="Best RF Model")
    else:
        mlflow.sklearn.save_model(best_model, "model",signature=signature)

    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")
# End MLFlow Experiments

InvalidParameterError: The 'scoring' parameter of GridSearchCV must be a str among {'normalized_mutual_info_score', 'f1_macro', 'neg_mean_absolute_percentage_error', 'r2', 'recall_samples', 'neg_median_absolute_error', 'completeness_score', 'jaccard_weighted', 'explained_variance', 'mutual_info_score', 'jaccard_macro', 'adjusted_mutual_info_score', 'jaccard_micro', 'accuracy', 'adjusted_rand_score', 'f1_samples', 'roc_auc_ovo', 'neg_root_mean_squared_log_error', 'roc_auc_ovo_weighted', 'recall_micro', 'average_precision', 'v_measure_score', 'positive_likelihood_ratio', 'roc_auc_ovr', 'd2_absolute_error_score', 'precision_micro', 'precision_macro', 'rand_score', 'precision_weighted', 'balanced_accuracy', 'recall_weighted', 'neg_negative_likelihood_ratio', 'recall_macro', 'recall', 'roc_auc_ovr_weighted', 'neg_root_mean_squared_error', 'max_error', 'precision', 'f1_weighted', 'top_k_accuracy', 'f1', 'homogeneity_score', 'neg_brier_score', 'f1_micro', 'neg_mean_squared_log_error', 'matthe